**Model Development**

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
train_df = pd.read_csv('gdrive/My Drive/CorporateBankrupt/training_data.csv')
test_df = pd.read_csv('gdrive/My Drive/CorporateBankrupt/test_data.csv')

In [8]:
train_df.head(2)

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,Attr11,Attr12,Attr13,Attr14,Attr15,Attr16,Attr17,Attr18,Attr19,Attr20,Attr21,Attr22,Attr23,Attr24,Attr25,Attr26,Attr27,Attr28,Attr29,Attr30,Attr31,Attr32,Attr33,Attr34,Attr35,Attr36,Attr37,Attr38,Attr39,Attr40,Attr41,Attr42,Attr43,Attr44,Attr45,Attr46,Attr47,Attr48,Attr49,Attr50,Attr51,Attr52,Attr53,Attr54,Attr55,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class
0,0.20055,0.37951,0.39641,2.0472,32.351,0.38825,0.24976,1.33050,1.1389,0.50494,0.24976,0.6598,0.16660,0.24976,497.42,0.73378,2.6349,0.24976,0.14942,43.370,1.2479,0.21402,0.11998,0.477060,0.50494,0.60411,1.4582,1.7615,5.9443,0.11788,0.14942,94.14,3.8772,0.56393,0.21402,1.7410,593.270000,0.50591,0.12804,0.662950,0.051402,0.12804,114.42,71.05,1.00970,1.5225,49.394,0.18530,0.11085,2.0420,0.37854,0.25792,2.2437,2.248,348690.0,0.12196,0.39718,0.87804,0.001924,8.4160,5.1372,82.658,4.4158,7.4277,0
1,0.20912,0.49988,0.47225,1.9447,14.786,0.00000,0.25834,0.99601,1.6996,0.49788,0.26114,0.5168,0.15835,0.25834,677.96,0.53838,2.0005,0.25834,0.15200,87.981,1.4293,0.24806,0.12304,0.276497,0.39542,0.43992,88.4440,16.9460,3.6884,0.26969,0.15200,122.17,2.9876,2.98760,0.20616,1.6996,114.742766,0.49788,0.12130,0.086422,0.064371,0.14595,199.49,111.51,0.51045,1.1252,100.130,0.23727,0.13961,1.9447,0.49988,0.33472,17.8660,17.866,2304.6,0.12130,0.42002,0.85300,0.000000,4.1486,3.2732,107.350,3.4000,60.9870,0


In [10]:
df = pd.concat([train_df,test_df],axis = 0)

In [11]:
X = df.drop(columns = ['class'])
y = df[['class']]

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [15]:
X_train.shape

(66102, 64)

In [16]:
X_test.shape

(16526, 64)

In [13]:
from sklearn.preprocessing import MinMaxScaler
mns = MinMaxScaler()

In [24]:
#fitting the scaler
mns.fit(X_train,y_train)

#transform training data
X_train_scaled = pd.DataFrame(mns.transform(X_train),columns = X_train.columns)

#transform test data
X_test_scaled = pd.DataFrame(mns.transform(X_test),columns = X_test.columns)

In [38]:
from sklearn.feature_selection import SelectKBest,chi2
bestfeatures = SelectKBest(score_func=chi2)

In [44]:
topf = bestfeatures.fit(X_train_scaled, y_train)

In [45]:
dfscores =pd.DataFrame(topf.scores_)
dfcolumns = pd.DataFrame(X_train_scaled.columns)

In [46]:
feature_scores = pd.concat([dfscores,dfcolumns],axis = 1)

In [47]:
feature_scores.columns = ['Score','Spec']

In [48]:
feature_scores

,Score,Spec
0,0.005584,Attr1
1,0.252359,Attr2
2,0.036163,Attr3
3,0.642949,Attr4
4,0.000211,Attr5
...,...,...
59,1.862349,Attr60
60,0.061763,Attr61
61,0.000210,Attr62
62,0.084451,Attr63


In [50]:
feature_scores.nlargest(n = 15,columns='Score')['Spec']

50    Attr51
28    Attr29
31    Attr32
16    Attr17
59    Attr60
49    Attr50
7      Attr8
46    Attr47
3      Attr4
19    Attr20
1      Attr2
51    Attr52
8      Attr9
54    Attr55
36    Attr37
Name: Spec, dtype: object

In [53]:
X_train = X_train[feature_scores.nlargest(n = 15,columns='Score')['Spec']]
X_test = X_test[feature_scores.nlargest(n = 15,columns='Score')['Spec']]

In [54]:
X_train.shape

(66102, 15)

# **Decision Tree Classifier**

In [212]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(ccp_alpha= 0.000154, max_depth= 21, criterion='entropy')

dtc.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.000154, class_weight=None,
                       criterion='entropy', max_depth=21, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [213]:
from sklearn.metrics import accuracy_score

In [214]:
y_pred = dtc.predict(X_test)

In [260]:
print("Training Score: {0}".format(round(accuracy_score(y_test,y_pred),2)))

Training Score: 0.84


In [261]:
print("Test Score: {0}".format(round(dtc.score(X_train,y_train),2)))

Test Score: 0.9


In [287]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.84      0.83      8104
           1       0.84      0.84      0.84      8422

    accuracy                           0.84     16526
   macro avg       0.84      0.84      0.84     16526
weighted avg       0.84      0.84      0.84     16526



# ***LightGBM Classifier***

In [228]:
from lightgbm import LGBMClassifier

In [255]:
lgb = LGBMClassifier(max_depth = 7,learning_rate= 0.5)

In [256]:
lgb.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.5, max_depth=7,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [263]:
y_pred2 = lgb.predict(X_test)

In [264]:
print("Training Score: {0}".format(round(accuracy_score(y_test,y_pred2),2)))

Training Score: 0.94


In [265]:
print("Test Score: {0}".format(round(lgb.score(X_train,y_train),2)))

Test Score: 0.97


In [286]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      8104
           1       0.96      0.92      0.94      8422

    accuracy                           0.94     16526
   macro avg       0.94      0.94      0.94     16526
weighted avg       0.94      0.94      0.94     16526



# ***MLP Classifier***

In [280]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(150,100,50), activation='relu', solver='adam', alpha=0.0005, batch_size='auto', learning_rate='constant', learning_rate_init=0.005, max_iter = 300)

In [281]:
mlp.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0005, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(150, 100, 50), learning_rate='constant',
              learning_rate_init=0.005, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [282]:
y_pred3 = mlp.predict(X_test)

In [283]:
print("Training Score: {0}".format(round(accuracy_score(y_test,y_pred3),2)))

Training Score: 0.59


In [284]:
print("Test Score: {0}".format(round(mlp.score(X_train,y_train),2)))

Test Score: 0.6


In [288]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

           0       0.55      0.93      0.69      8104
           1       0.79      0.26      0.40      8422

    accuracy                           0.59     16526
   macro avg       0.67      0.60      0.54     16526
weighted avg       0.67      0.59      0.54     16526

